<a href="https://colab.research.google.com/github/tomonari-masada/course2025-nlp/blob/main/08_token_by_token_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_id = "LiquidAI/LFM2-1.2B"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    dtype="bfloat16",
#    attn_implementation="flash_attention_2",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# Generate answer
prompt = "What is C. elegans?"
input_ids = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=True,
).to(model.device)

output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.3,
    min_p=0.15,
    repetition_penalty=1.05,
    max_new_tokens=512,
)

print(tokenizer.decode(output[0], skip_special_tokens=False))

# <|startoftext|><|im_start|>user
# What is C. elegans?<|im_end|>
# <|im_start|>assistant
# C. elegans, also known as Caenorhabditis elegans, is a small, free-living
# nematode worm (roundworm) that belongs to the phylum Nematoda.


<|startoftext|><|im_start|>user
What is C. elegans?<|im_end|>
<|im_start|>assistant
C. elegans, also known as Caenorhabditis elegans, is a small, free-living nematode worm (roundworm) that belongs to the phylum Nematoda. It's one of the simplest and most well-studied organisms in scientific research, particularly in the fields of genetics, development, and neuroscience. Native to Japan, C. elegans measures about 0.2 millimeters in length and has a translucent body with a distinctive segmented appearance.

As a model organism, C. elegans has been extensively used in scientific research due to its relatively short lifespan (about 2-4 weeks under optimal conditions), simple nervous system, and ease of cultivation in laboratory settings. Its genome was first sequenced in 1998, making it an ideal organism for genetic studies. Researchers use C. elegans to investigate fundamental biological processes, such as cell division, neural development, and aging, which have implications for understan

In [ ]:
prompt = "What is C. elegans?"
input_ids = tokenizer.apply_chat_template(
    [{"role": "user", "content": prompt}],
    add_generation_prompt=True,
    return_tensors="pt",
    tokenize=True,
).to(model.device)

In [ ]:
answer = ""
for _ in range(100):
    inputs_embeds = model.get_input_embeddings()(input_ids)
    outputs = model(
        attention_mask=None,
        inputs_embeds=inputs_embeds,
        use_cache=model.config.use_cache,
    )
    next_token = tokenizer.decode(outputs.logits[0,-1,:].argmax(dim=-1), skip_special_tokens=False)
    print(f"subword [{next_token}]")
    answer += next_token
    input_ids = torch.cat((input_ids, outputs.logits[:,-1:,:].argmax(dim=-1)), dim=-1)
    if next_token == tokenizer.eos_token:
        break
print(answer)

subword [C]
subword [.]
subword [ eleg]
subword [ans]
subword [,]
subword [ also]
subword [ known]
subword [ as]
subword [ Ca]
subword [en]
subword [or]
subword [hab]
subword [d]
subword [itis]
subword [ eleg]
subword [ans]
subword [,]
subword [ is]
subword [ a]
subword [ small]
subword [,]
subword [ free]
subword [-l]
subword [iving]
subword [ nemat]
subword [ode]
subword [ worm]
subword [ (]
subword [round]
subword [worm]
subword [)]
subword [ that]
subword [ belongs]
subword [ to]
subword [ the]
subword [ ph]
subword [ylum]
subword [ N]
subword [emat]
subword [oda]
subword [.]
subword [ It]
subword [ is]
subword [ a]
subword [ model]
subword [ organism]
subword [ widely]
subword [ used]
subword [ in]
subword [ scientific]
subword [ research]
subword [,]
subword [ particularly]
subword [ in]
subword [ the]
subword [ fields]
subword [ of]
subword [ genetics]
subword [,]
subword [ development]
subword [,]
subword [ neuroscience]
subword [,]
subword [ and]
subword [ aging]
subword [.]
s